In [1]:
import sys

# in order to import the modules located at the root directory
sys.path.append("..")

In [2]:
import logging
from importlib import reload  # Not needed in Python 2

reload(logging)
logging.basicConfig(
    format="%(message)s",
    level=logging.DEBUG,
)

LOGGER = logging.getLogger(__name__)

In [3]:
from mrs.dataset import SessionBuilder, DATA_PATH

builder = SessionBuilder(style="formal")
sessions = builder.build_sessions(data_path=DATA_PATH)
utts = builder.get_utterances(sessions)

Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.
Number of sessions: 236


In [6]:
train_sessions = sessions[:-15]
valid_sessions = sessions[-15:]
print(len(train_sessions), len(valid_sessions))

221 15


In [8]:
print(train_sessions[0].conv)

['안녕하세요. 저는 고양이 6마리 키워요.', '고양이를 6마리나요? 키우는거 안 힘드세요?', '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.', '가장 나이가 많은 고양이가 어떻게 돼요?', '여섯 살입니다. 갈색 고양이에요.', '그럼 가장 어린 고양이가 어떻게 돼요?', '한 살입니다. 작년에 분양 받았어요.', '그럼 고양이들끼리 안 싸우나요?', '저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.']


In [14]:
# make positive/negative pairs
# data augmentation
import random
from collections import defaultdict
from typing import List
from mrs.dataset import Session


def build_finetune_dataset(
    utts: List[str], sessions: List[Session], n_turns: int = 5, n_negs: int = 4
) -> dict:
    data_json = defaultdict(dict)

    cnt = 0
    for session in sessions:
        ctx = [session.conv[0]]
        for turn in range(1, len(session.conv)):
            utt = session.conv[turn]
            neg_candidates = random.sample(utts, n_negs)
            data_json[cnt]["context"] = ctx[:][-n_turns:]
            data_json[cnt]["positive_response"] = utt
            data_json[cnt]["negative_responses"] = neg_candidates
            ctx.append(utt)
            cnt += 1

    return data_json


train_json = build_finetune_dataset(utts=utts, sessions=train_sessions)
valid_json = build_finetune_dataset(utts=utts, sessions=valid_sessions)
print(len(train_json), len(valid_json))

3020 214


In [18]:
train_json[0]

{'context': ['안녕하세요. 저는 고양이 6마리 키워요.'],
 'positive_response': '고양이를 6마리나요? 키우는거 안 힘드세요?',
 'negative_responses': ['네, 어느 순간부터 일과가 되었어요.',
  '어떤 쇼를 가장 좋아하시나요?',
  '당장 다음주에 무도회를 나가게 되는데, 어떻게 할 지 모르겠어요.',
  '태어날 때 부터 색맹이였어요.']}